In [36]:
def initialize_metrics_dictionary():
    metrics = {
        'Edges': {
            'Trade Profitability': None,
            'Immediacy of Threat': None,
            'Board Control Contribution': None,
            'Edge Color': None,
            'Type of Interaction': None,
            'Opening/Midgame/Endgame Relevance': None,
            'Risk Factor': None,
            'Edge Direction': None,
            'Tactical Complexity': None,
            'Historical Popularity': None,
            'Edge Texture': None,
            'Game Phase Relevance': None,
            'Depth of Calculation': None,
            'Edge Animation': None
        },
        'Nodes': {
            'Node Size': {
                'Piece Value': None,
                'Strategic Importance': None,
                'King Safety': None
            },
            'Node Color': {
                'Piece Color': None,
                'Defender\'s Value': None,
                'Threat Multiplier': None
            },
            'Node Shape': {
                'Type of Piece': None
            },
            'Node Texture': {
                'King Safety': None,
                'Depth of Calculation': None
            },
            'Node Label': {
                'Piece Identifier': None,
                'Tactical Opportunities': None
            },
            'Node Border': {
                'Number of Defenders': None,
                'Number of Attackers': None
            },
            'Node Animation': {
                'Immediacy of Threat to the Node': None,
                'Tactical Opportunities': None
            }
        }
    }
    return metrics


In [37]:
import chess

def identify_game_phase(board):
    # Count the number of each type of piece on the board
    piece_counts = {
        'P': len(board.pieces(chess.PAWN, chess.WHITE)),
        'p': len(board.pieces(chess.PAWN, chess.BLACK)),
        'N': len(board.pieces(chess.KNIGHT, chess.WHITE)),
        'n': len(board.pieces(chess.KNIGHT, chess.BLACK)),
        'B': len(board.pieces(chess.BISHOP, chess.WHITE)),
        'b': len(board.pieces(chess.BISHOP, chess.BLACK)),
        'R': len(board.pieces(chess.ROOK, chess.WHITE)),
        'r': len(board.pieces(chess.ROOK, chess.BLACK)),
        'Q': len(board.pieces(chess.QUEEN, chess.WHITE)),
        'q': len(board.pieces(chess.QUEEN, chess.BLACK)),
    }

    # Calculate total material for both sides
    total_material_white = (
        piece_counts['P'] * 1 +
        piece_counts['N'] * 3 +
        piece_counts['B'] * 3 +
        piece_counts['R'] * 5 +
        piece_counts['Q'] * 9
    )

    total_material_black = (
        piece_counts['p'] * 1 +
        piece_counts['n'] * 3 +
        piece_counts['b'] * 3 +
        piece_counts['r'] * 5 +
        piece_counts['q'] * 9
    )

    # Identify game phase based on remaining material
    if total_material_white + total_material_black > 50:
        return "Opening"
    elif total_material_white + total_material_black > 20:
        return "Midgame"
    else:
        return "Endgame"

# Loading a position from a FEN string
board = chess.Board("8/Rp4pp/6k1/2B5/2b1p3/8/P1P2PPP/4K3 b - - 0 26")

# Test the function
print("Game Phase:", identify_game_phase(board))

Game Phase: Endgame


In [38]:
import chess

def calculate_key_square_control(board):
    # Define key squares
    key_squares = [chess.D4, chess.D5, chess.E4, chess.E5]
    
    # Initialize control counts
    control = {
        'white': {square: 0 for square in key_squares},
        'black': {square: 0 for square in key_squares},
    }
    
    # Calculate control for each key square
    for square in key_squares:
        control['white'][square] = len(board.attackers(chess.WHITE, square))
        control['black'][square] = len(board.attackers(chess.BLACK, square))
        
    return control

# Test the function
board = chess.Board()
print("Initial Key Square Control:", calculate_key_square_control(board))

# Loading a position from a FEN string
board = chess.Board("8/Rp4pp/6k1/2B5/2b1p3/8/P1P2PPP/4K3 b - - 0 26")

print("Key Square Control After Moves:", calculate_key_square_control(board))

Initial Key Square Control: {'white': {27: 0, 35: 0, 28: 0, 36: 0}, 'black': {27: 0, 35: 0, 28: 0, 36: 0}}
Key Square Control After Moves: {'white': {27: 1, 35: 0, 28: 0, 36: 0}, 'black': {27: 0, 35: 1, 28: 0, 36: 0}}


In [39]:
import chess
import chess.pgn
import numpy as np
import pandas as pd

def get_piece_value(piece):
    piece_values = {
        'p': 1, 'P': 1,
        'n': 3, 'N': 3,
        'b': 3, 'B': 3,
        'r': 5, 'R': 5,
        'q': 9, 'Q': 9,
        'k': 0, 'K': 0
    }
    return piece_values.get(piece.symbol(), 0)

def get_piece_names(board):
    return [str(board.piece_at(i)) + chess.square_name(i) for i in range(64) if board.piece_at(i) is not None]

def initialize_adj_matrix(piece_names):
    matrix = np.zeros((len(piece_names), len(piece_names)))
    return pd.DataFrame(matrix, columns=piece_names, index=piece_names)

def update_adj_matrix(adj_matrix, board):
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece is not None:
            piece_name = str(piece) + chess.square_name(square)
            attackers = board.attackers(chess.WHITE if piece.color == chess.BLACK else chess.BLACK, square)
            defenders = board.attackers(piece.color, square)
            
            for attacker in attackers:
                attacker_piece = board.piece_at(attacker)
                if attacker_piece is not None:
                    attacker_name = str(attacker_piece) + chess.square_name(attacker)
                    adj_matrix.loc[piece_name, attacker_name] = -get_piece_value(attacker_piece)
            
            for defender in defenders:
                defender_piece = board.piece_at(defender)
                if defender_piece is not None:
                    defender_name = str(defender_piece) + chess.square_name(defender)
                    adj_matrix.loc[piece_name, defender_name] = get_piece_value(defender_piece)
                    
def get_fen_list_from_pgn_file(pgn_file_path):
    with open(pgn_file_path, 'r') as pgn:
        game = chess.pgn.read_game(pgn)
    fen_list = []
    board = game.board()
    for move in game.mainline_moves():
        board.push(move)
        fen_str = board.fen()
        fen_list.append(fen_str)
    return fen_list

def board_structures(fen_list):
    adj_matrices = []
    for fen in fen_list:
        board = chess.Board(fen)
        piece_names = [str(board.piece_at(i)) + chess.square_name(i) for i in range(64) if board.piece_at(i) is not None]
        
        # Initialize adjacency matrix
        matrix = np.zeros((len(piece_names), len(piece_names)))
        adj_matrix = pd.DataFrame(matrix, columns=piece_names, index=piece_names)
        
        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece is not None:
                piece_name = str(piece) + chess.square_name(square)
                
                # Get attackers and defenders for the current square
                attackers = board.attackers(not piece.color, square)
                defenders = board.attackers(piece.color, square)
                
                for attacker in attackers:
                    attacker_piece = board.piece_at(attacker)
                    if attacker_piece is not None:
                        attacker_name = str(attacker_piece) + chess.square_name(attacker)
                        adj_matrix.loc[piece_name, attacker_name] = -get_piece_value(attacker_piece)
                
                for defender in defenders:
                    defender_piece = board.piece_at(defender)
                    if defender_piece is not None:
                        defender_name = str(defender_piece) + chess.square_name(defender)
                        adj_matrix.loc[piece_name, defender_name] = get_piece_value(defender_piece)
        
        adj_matrix = adj_matrix.fillna(0).astype(int)
        adj_matrix.attrs['fen'] = fen
        adj_matrices.append(adj_matrix)
    return adj_matrices


# Test the function
pgn_file_path = '/Users/tamasmakos/dev/chess-graph-analysis/tamasmakos_vs_cartierer_2023.10.06.pgn'  # Replace with your actual PGN file path
fen_list = get_fen_list_from_pgn_file(pgn_file_path)
adj_matrices = board_structures(fen_list)
print(adj_matrices[30].shape)

(23, 23)


In [15]:
import chess

def get_piece_value(piece):
    piece_values = {
        'p': 1, 'P': 1,
        'n': 3, 'N': 3,
        'b': 3, 'B': 3,
        'r': 5, 'R': 5,
        'q': 9, 'Q': 9,
        'k': 0, 'K': 0  # King is usually not assigned a value
    }
    return piece_values.get(piece.symbol(), 0)

def calculate_trade_profitability(fen):
    board = chess.Board(fen)
    trade_profitability = {}
    
    # Step 1: Check whose turn it is
    current_turn = board.turn
    
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        
        # Step 2: Check if there are any attacking opportunities
        if piece and piece.color != current_turn:
            attackers = board.attackers(current_turn, square)
            
            # Step 3: Calculate trading_profitability
            for attacker in attackers:
                attacker_piece = board.piece_at(attacker)
                
                # Step 4: If the attacking piece is pinned to the king, continue
                if board.is_pinned(current_turn, attacker):
                    continue
                
                # Step 5: If the piece is pinned to a higher value piece, get the score of the higher value piece
                pinned_value = 0
                
                # Step 6: The calculation
                if pinned_value > 0:
                    profitability = get_piece_value(piece) - pinned_value
                else:
                    profitability = get_piece_value(piece) - get_piece_value(attacker_piece)
                
                # New Step: Check if capturing leaves a higher-value piece vulnerable
                board.push(chess.Move(attacker, square))  # Make the move temporarily
                
                for higher_value_square in chess.SQUARES:
                    higher_value_piece = board.piece_at(higher_value_square)
                    if higher_value_piece and higher_value_piece.color == current_turn:
                        higher_value_attackers = board.attackers(not current_turn, higher_value_square)
                        
                        if higher_value_attackers:
                            # Check if the piece was already under attack before the move
                            board.pop()  # Revert the move to check the original position
                            original_attackers = board.attackers(not current_turn, higher_value_square)
                            
                            if not original_attackers:
                                # The piece became vulnerable due to the move, adjust profitability
                                profitability -= get_piece_value(higher_value_piece)
                            
                            board.push(chess.Move(attacker, square))  # Reapply the move to continue checking
                
                board.pop()  # Revert the move
                
                trade_key = f"{attacker_piece.symbol()} at {chess.square_name(attacker)} -> {piece.symbol()} at {chess.square_name(square)}"
                trade_profitability[trade_key] = profitability
    
    return trade_profitability

# Test the function
fen = "r2qk2r/p1pbppbp/1p3Bpn/3p4/P2Q4/1P6/2P1PPPP/RN2KBNR w KQkq - 1 9"  # Replace with your actual FEN string
result = calculate_trade_profitability(fen)
print(result)


{'Q at d4 -> p at d5': -8, 'Q at d4 -> p at b6': -8, 'B at f6 -> p at e7': -11, 'B at f6 -> b at g7': 0}
